In [95]:
# choose, how werbose answer is expected
VERBOSE = "SIMPLE" # SIMPLE, DEBUG

PROJECT_ID = "osa-hy-project"
REGION = "europe-central2"

ME_REGION = "europe-central2"
ME_INDEX_NAME = f"{PROJECT_ID}-me-index"
ME_EMBEDDING_DIR = f"{PROJECT_ID}-me-bucket"
ME_DIMENSIONS = 768  # when using Vertex PaLM Embedding

# index and endpoint deployed in Vertex AI for PROJECT_ID
ME_INDEX_ID = ""
ME_INDEX_ENDPOINT_ID = ""

In [ ]:
# Install Vertex AI LLM SDK
! pip install --user --upgrade google-cloud-aiplatform==1.31.0 langchain==0.0.201

# For Matching Engine integration dependencies (default embeddings)
! pip install --user tensorflow_hub==0.13.0 tensorflow_text==2.12.1

In [96]:
import json
import textwrap
# Utils
import time
import uuid
from typing import List

import numpy as np
import vertexai
# Vertex AI
from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Langchain
import langchain
print(f"LangChain version: {langchain.__version__}")

from langchain.chains import RetrievalQA
from langchain.document_loaders import GCSDirectoryLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

#add memory to conversation
from langchain.memory import ConversationBufferMemory

Vertex AI SDK version: 1.31.0
LangChain version: 0.0.201


In [19]:
"""Vertex Matching Engine implementation of the vector store."""
from __future__ import annotations

import logging
import uuid
from typing import Any, Iterable, List, Optional, Type

from langchain.docstore.document import Document
from langchain.embeddings import TensorflowHubEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.vectorstores.base import VectorStore

from google.cloud import storage
from google.cloud.aiplatform import MatchingEngineIndex, MatchingEngineIndexEndpoint
from google.cloud import aiplatform_v1
from google.oauth2.service_account import Credentials
import google.auth
import google.auth.transport.requests

logger = logging.getLogger()


class MatchingEngine(VectorStore):
    """Vertex Matching Engine implementation of the vector store.

    While the embeddings are stored in the Matching Engine, the embedded
    documents will be stored in GCS.

    An existing Index and corresponding Endpoint are preconditions for
    using this module.

    See usage in docs/modules/indexes/vectorstores/examples/matchingengine.ipynb

    Note that this implementation is mostly meant for reading if you are
    planning to do a real time implementation. While reading is a real time
    operation, updating the index takes close to one hour."""

    def __init__(
        self,
        project_id: str,
        region: str,
        index: MatchingEngineIndex,
        endpoint: MatchingEngineIndexEndpoint,
        embedding: Embeddings,
        gcs_client: storage.Client,
        index_client: aiplatform_v1.IndexServiceClient,
        index_endpoint_client: aiplatform_v1.IndexEndpointServiceClient,
        gcs_bucket_name: str,
        credentials: Credentials = None,
    ):
        """Vertex Matching Engine implementation of the vector store.

        While the embeddings are stored in the Matching Engine, the embedded
        documents will be stored in GCS.

        An existing Index and corresponding Endpoint are preconditions for
        using this module.

        See usage in
        docs/modules/indexes/vectorstores/examples/matchingengine.ipynb.

        Note that this implementation is mostly meant for reading if you are
        planning to do a real time implementation. While reading is a real time
        operation, updating the index takes close to one hour.

        Attributes:
            project_id: The GCS project id.
            index: The created index class. See
            ~:func:`MatchingEngine.from_components`.
            endpoint: The created endpoint class. See
            ~:func:`MatchingEngine.from_components`.
            embedding: A :class:`Embeddings` that will be used for
            embedding the text sent. If none is sent, then the
            multilingual Tensorflow Universal Sentence Encoder will be used.
            gcs_client: The Google Cloud Storage client.
            credentials (Optional): Created GCP credentials.
        """
        super().__init__()
        self._validate_google_libraries_installation()

        self.project_id = project_id
        self.region = region
        self.index = index
        self.endpoint = endpoint
        self.embedding = embedding
        self.gcs_client = gcs_client
        self.index_client = index_client
        self.index_endpoint_client = index_endpoint_client
        self.gcs_client = gcs_client
        self.credentials = credentials
        self.gcs_bucket_name = gcs_bucket_name
        logger.info("local helper functions used.")

    def _validate_google_libraries_installation(self) -> None:
        """Validates that Google libraries that are needed are installed."""
        try:
            from google.cloud import aiplatform, storage  # noqa: F401
            from google.oauth2 import service_account  # noqa: F401
        except ImportError:
            raise ImportError(
                "You must run `pip install --upgrade "
                "google-cloud-aiplatform google-cloud-storage`"
                "to use the MatchingEngine Vectorstore."
            )

    def add_texts(
        self,
        texts: Iterable[str],
        metadatas: Optional[List[dict]] = None,
        **kwargs: Any,
    ) -> List[str]:
        """Run more texts through the embeddings and add to the vectorstore.

        Args:
            texts: Iterable of strings to add to the vectorstore.
            metadatas: Optional list of metadatas associated with the texts.
            kwargs: vectorstore specific parameters.

        Returns:
            List of ids from adding the texts into the vectorstore.
        """
        logger.info("Embedding documents.")
        embeddings = self.embedding.embed_documents(list(texts))
        insert_datapoints_payload = []
        ids = []

        # Streaming index update
        for idx, (embedding, text, metadata) in enumerate(
            zip(embeddings, texts, metadatas)
        ):
            id = uuid.uuid4()
            ids.append(id)
            self._upload_to_gcs(text, f"documents/{id}")
            metadatas[idx]
            insert_datapoints_payload.append(
                aiplatform_v1.IndexDatapoint(
                    datapoint_id=str(id),
                    feature_vector=embedding,
                    restricts=metadata if metadata else [],
                )
            )
            if idx % 100 == 0:
                upsert_request = aiplatform_v1.UpsertDatapointsRequest(
                    index=self.index.name, datapoints=insert_datapoints_payload
                )
                response = self.index_client.upsert_datapoints(request=upsert_request)
                insert_datapoints_payload = []
        if len(insert_datapoints_payload) > 0:
            upsert_request = aiplatform_v1.UpsertDatapointsRequest(
                index=self.index.name, datapoints=insert_datapoints_payload
            )
            _ = self.index_client.upsert_datapoints(request=upsert_request)

        logger.info("Updated index with new configuration.")
        logger.info(f"Indexed {len(ids)} documents to Matching Engine.")

        return ids

    def _upload_to_gcs(self, data: str, gcs_location: str) -> None:
        """Uploads data to gcs_location.

        Args:
            data: The data that will be stored.
            gcs_location: The location where the data will be stored.
        """
        bucket = self.gcs_client.get_bucket(self.gcs_bucket_name)
        blob = bucket.blob(gcs_location)
        blob.upload_from_string(data)

    def get_matches(
        self,
        embeddings: List[str],
        n_matches: int,
        index_endpoint: MatchingEngineIndexEndpoint,
    ) -> str:
        """
        get matches from matching engine given a vector query
        Uses public endpoint

        """
        import requests
        import json

        request_data = {
            "deployed_index_id": index_endpoint.deployed_indexes[0].id,
            "return_full_datapoint": True,
            "queries": [
                {
                    "datapoint": {"datapoint_id": f"{i}", "feature_vector": emb},
                    "neighbor_count": n_matches,
                }
                for i, emb in enumerate(embeddings)
            ],
        }

        endpoint_address = self.endpoint.public_endpoint_domain_name
        rpc_address = f"https://{endpoint_address}/v1beta1/{index_endpoint.resource_name}:findNeighbors"
        endpoint_json_data = json.dumps(request_data)

        logger.debug(f"Querying Matching Engine Index Endpoint {rpc_address}")

        request = google.auth.transport.requests.Request()
        self.credentials.refresh(request)
        header = {"Authorization": "Bearer " + self.credentials.token}

        return requests.post(rpc_address, data=endpoint_json_data, headers=header)

    def similarity_search(
        self, query: str, k: int = 4, search_distance: float = 0.65, **kwargs: Any
    ) -> List[Document]:
        """Return docs most similar to query.

        Args:
            query: The string that will be used to search for similar documents.
            k: The amount of neighbors that will be retrieved.
            search_distance: filter search results by  search distance by adding a threshold value

        Returns:
            A list of k matching documents.
        """

        logger.info(f"Embedding query {query}.")
        embedding_query = self.embedding.embed_documents([query])
        deployed_index_id = self._get_index_id()
        logger.info(f"Deployed Index ID = {deployed_index_id}")

        # TO-DO: Pending query sdk integration
        # response = self.endpoint.match(
        #     deployed_index_id=self._get_index_id(),
        #     queries=embedding_query,
        #     num_neighbors=k,
        # )

        response = self.get_matches(embedding_query, k, self.endpoint)

        if response.status_code == 200:
            response = response.json()["nearestNeighbors"]
        else:
            raise Exception(f"Failed to query index {str(response)}")

        if len(response) == 0:
            return []

        logger.info(f"Found {len(response)} matches for the query {query}.")

        results = []

        # I'm only getting the first one because queries receives an array
        # and the similarity_search method only recevies one query. This
        # means that the match method will always return an array with only
        # one element.
        for doc in response[0]["neighbors"]:
            page_content = self._download_from_gcs(
                f"documents/{doc['datapoint']['datapointId']}"
            )
            metadata = {}
            if "restricts" in doc["datapoint"]:
                metadata = {
                    item["namespace"]: item["allowList"][0]
                    for item in doc["datapoint"]["restricts"]
                }
            if "distance" in doc:
                metadata["score"] = doc["distance"]
                if doc["distance"] >= search_distance:
                    results.append(
                        Document(page_content=page_content, metadata=metadata)
                    )
            else:
                results.append(Document(page_content=page_content, metadata=metadata))

        logger.info("Downloaded documents for query.")

        return results

    def _get_index_id(self) -> str:
        """Gets the correct index id for the endpoint.

        Returns:
            The index id if found (which should be found) or throws
            ValueError otherwise.
        """
        for index in self.endpoint.deployed_indexes:
            if index.index == self.index.name:
                return index.id

        raise ValueError(
            f"No index with id {self.index.name} "
            f"deployed on enpoint "
            f"{self.endpoint.display_name}."
        )

    def _download_from_gcs(self, gcs_location: str) -> str:
        """Downloads from GCS in text format.

        Args:
            gcs_location: The location where the file is located.

        Returns:
            The string contents of the file.
        """
        bucket = self.gcs_client.get_bucket(self.gcs_bucket_name)
        try:
            blob = bucket.blob(gcs_location)
            return blob.download_as_string()
        except Exception:
            return ""

    @classmethod
    def from_texts(
        cls: Type["MatchingEngine"],
        texts: List[str],
        embedding: Embeddings,
        metadatas: Optional[List[dict]] = None,
        **kwargs: Any,
    ) -> "MatchingEngine":
        """Use from components instead."""
        raise NotImplementedError(
            "This method is not implemented. Instead, you should initialize the class"
            " with `MatchingEngine.from_components(...)` and then call "
            "`from_texts`"
        )

    @classmethod
    def from_documents(
        cls: Type["MatchingEngine"],
        documents: List[str],
        embedding: Embeddings,
        metadatas: Optional[List[dict]] = None,
        **kwargs: Any,
    ) -> "MatchingEngine":
        """Use from components instead."""
        raise NotImplementedError(
            "This method is not implemented. Instead, you should initialize the class"
            " with `MatchingEngine.from_components(...)` and then call "
            "`from_documents`"
        )

    @classmethod
    def from_components(
        cls: Type["MatchingEngine"],
        project_id: str,
        region: str,
        gcs_bucket_name: str,
        index_id: str,
        endpoint_id: str,
        credentials_path: Optional[str] = None,
        embedding: Optional[Embeddings] = None,
    ) -> "MatchingEngine":
        """Takes the object creation out of the constructor.

        Args:
            project_id: The GCP project id.
            region: The default location making the API calls. It must have
            the same location as the GCS bucket and must be regional.
            gcs_bucket_name: The location where the vectors will be stored in
            order for the index to be created.
            index_id: The id of the created index.
            endpoint_id: The id of the created endpoint.
            credentials_path: (Optional) The path of the Google credentials on
            the local file system.
            embedding: The :class:`Embeddings` that will be used for
            embedding the texts.

        Returns:
            A configured MatchingEngine with the texts added to the index.
        """
        gcs_bucket_name = cls._validate_gcs_bucket(gcs_bucket_name)

        # Set credentials
        if credentials_path:
            credentials = cls._create_credentials_from_file(credentials_path)
        else:
            credentials, _ = google.auth.default()
            request = google.auth.transport.requests.Request()
            credentials.refresh(request)

        index = cls._create_index_by_id(index_id, project_id, region, credentials)
        endpoint = cls._create_endpoint_by_id(
            endpoint_id, project_id, region, credentials
        )

        gcs_client = cls._get_gcs_client(credentials, project_id)
        index_client = cls._get_index_client(project_id, region, credentials)
        index_endpoint_client = cls._get_index_endpoint_client(
            project_id, region, credentials
        )
        cls._init_aiplatform(project_id, region, gcs_bucket_name, credentials)

        return cls(
            project_id=project_id,
            region=region,
            index=index,
            endpoint=endpoint,
            embedding=embedding or cls._get_default_embeddings(),
            gcs_client=gcs_client,
            index_client=index_client,
            index_endpoint_client=index_endpoint_client,
            credentials=credentials,
            gcs_bucket_name=gcs_bucket_name,
        )

    @classmethod
    def _validate_gcs_bucket(cls, gcs_bucket_name: str) -> str:
        """Validates the gcs_bucket_name as a bucket name.

        Args:
              gcs_bucket_name: The received bucket uri.

        Returns:
              A valid gcs_bucket_name or throws ValueError if full path is
              provided.
        """
        gcs_bucket_name = gcs_bucket_name.replace("gs://", "")
        if "/" in gcs_bucket_name:
            raise ValueError(
                f"The argument gcs_bucket_name should only be "
                f"the bucket name. Received {gcs_bucket_name}"
            )
        return gcs_bucket_name

    @classmethod
    def _create_credentials_from_file(
        cls, json_credentials_path: Optional[str]
    ) -> Optional[Credentials]:
        """Creates credentials for GCP.

        Args:
             json_credentials_path: The path on the file system where the
             credentials are stored.

         Returns:
             An optional of Credentials or None, in which case the default
             will be used.
        """

        from google.oauth2 import service_account

        credentials = None
        if json_credentials_path is not None:
            credentials = service_account.Credentials.from_service_account_file(
                json_credentials_path
            )

        return credentials

    @classmethod
    def _create_index_by_id(
        cls, index_id: str, project_id: str, region: str, credentials: "Credentials"
    ) -> MatchingEngineIndex:
        """Creates a MatchingEngineIndex object by id.

        Args:
            index_id: The created index id.

        Returns:
            A configured MatchingEngineIndex.
        """

        from google.cloud import aiplatform_v1

        logger.debug(f"Creating matching engine index with id {index_id}.")
        index_client = cls._get_index_client(project_id, region, credentials)
        request = aiplatform_v1.GetIndexRequest(name=index_id)
        return index_client.get_index(request=request)

    @classmethod
    def _create_endpoint_by_id(
        cls, endpoint_id: str, project_id: str, region: str, credentials: "Credentials"
    ) -> MatchingEngineIndexEndpoint:
        """Creates a MatchingEngineIndexEndpoint object by id.

        Args:
            endpoint_id: The created endpoint id.

        Returns:
            A configured MatchingEngineIndexEndpoint.
            :param project_id:
            :param region:
            :param credentials:
        """

        from google.cloud import aiplatform

        logger.debug(f"Creating endpoint with id {endpoint_id}.")
        return aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name=endpoint_id,
            project=project_id,
            location=region,
            credentials=credentials,
        )

    @classmethod
    def _get_gcs_client(
        cls, credentials: "Credentials", project_id: str
    ) -> "storage.Client":
        """Lazily creates a GCS client.

        Returns:
            A configured GCS client.
        """

        from google.cloud import storage

        return storage.Client(credentials=credentials, project=project_id)

    @classmethod
    def _get_index_client(
        cls, project_id: str, region: str, credentials: "Credentials"
    ) -> "storage.Client":
        """Lazily creates a Matching Engine Index client.

        Returns:
            A configured Matching Engine Index client.
        """

        from google.cloud import aiplatform_v1

        # PARENT = f"projects/{project_id}/locations/{region}"
        ENDPOINT = f"{region}-aiplatform.googleapis.com"
        return aiplatform_v1.IndexServiceClient(
            client_options=dict(api_endpoint=ENDPOINT), credentials=credentials
        )

    @classmethod
    def _get_index_endpoint_client(
        cls, project_id: str, region: str, credentials: "Credentials"
    ) -> "storage.Client":
        """Lazily creates a Matching Engine Index Endpoint client.

        Returns:
            A configured Matching Engine Index Endpoint client.
        """

        from google.cloud import aiplatform_v1

        # PARENT = f"projects/{project_id}/locations/{region}"
        ENDPOINT = f"{region}-aiplatform.googleapis.com"
        return aiplatform_v1.IndexEndpointServiceClient(
            client_options=dict(api_endpoint=ENDPOINT), credentials=credentials
        )

    @classmethod
    def _init_aiplatform(
        cls,
        project_id: str,
        region: str,
        gcs_bucket_name: str,
        credentials: "Credentials",
    ) -> None:
        """Configures the aiplatform library.

        Args:
            project_id: The GCP project id.
            region: The default location making the API calls. It must have
            the same location as the GCS bucket and must be regional.
            gcs_bucket_name: GCS staging location.
            credentials: The GCS Credentials object.
        """

        from google.cloud import aiplatform

        logger.debug(
            f"Initializing AI Platform for project {project_id} on "
            f"{region} and for {gcs_bucket_name}."
        )
        aiplatform.init(
            project=project_id,
            location=region,
            staging_bucket=gcs_bucket_name,
            credentials=credentials,
        )

    @classmethod
    def _get_default_embeddings(cls) -> TensorflowHubEmbeddings:
        """This function returns the default embedding."""
        return TensorflowHubEmbeddings()

In [84]:
# Text model instance integrated with langChain
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=512,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=False,
)

# Embeddings API integrated with langChain (initially EMBEDDING_QPM=100 and EMBEDDING_NUM_BATCH=5)
EMBEDDING_QPM = 50
EMBEDDING_NUM_BATCH = 4
embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko-multilingual",
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

# initialize vector store
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=ME_REGION,
    gcs_bucket_name=f"gs://{ME_EMBEDDING_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID,
)

# Create chain to answer questions
NUMBER_OF_RESULTS = 10
SEARCH_DISTANCE_THRESHOLD = 0.65

# Expose index to the retriever
retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
)

In [85]:
# test

# Test whether search from vector store is working
me.similarity_search("jakie są nawozy do rzepaku?", k=2)

[Document(page_content='Nawozy jedno i wieloskładnikowe\n\nKażda z roślin uprawnych posiada dość specyficzne potrzeby pokarmowe w stosunku do makro- i mikroelementów. Oznacza to, że w agrotechnice uprawy należy uwzględnić oprócz nawożenia uniwersalnymi nawozami dolistnymi także aplikację dedykowanych produktów. Preparaty te zawierają wysoką koncentrację składników kluczowych dla danej rośliny uprawnej np. dla rzepaku ozimego może być to bor, natomiast dla kukurydzy cynk.\n\nTermin aplikacji nawozów dolistnych', metadata={'source': 'osa-hy-project-documents/documents/osa-txt', 'document_name': 'oferta-srodki-ochrony-roslin-nawozy.html', 'chunk': '58', 'score': 0.8309053182601929}),
 Document(page_content='Ilość zarejestrowanych herbicydów w uprawie kukurydzy umożliwia aplikację środków w bardzo szerokim zakresie faz rozwojowych. Począwszy od zabiegów przed wschodami, a kończąc na fazie 9-liści kukurydzy. Praktyka rolnicza i liczne doniesienia naukowe wskazują, że oprysk na chwasty należ

In [86]:
template = """SYSTEM: Dzień dobry. Jestem pomocnym agentem, który pomaga w odszukaniu informacji na zadane pytanie: {question}.

Jako asystent mogę używać TYLKO i wyłącznie kontekstu zawartego pomiędzy znacznikami <CONTEXT></CONTEXT> aby odpowiadać na pytania. 

Nie generuję odpowiedzi dla poniższych przypadków:
 - jeśli odpowiedź na pytanie nie może zostać udzielona wyłącznie na podstawie kontekstu, odpowiadam "nie posiadam informacji aby udzielić odpowiedzi na to pytanie."
 - jeśli kontekst jest pusty odpowiadam "nie znam odpowedzi."

================
<CONTEXT>{context}</CONTEXT>
================
<CHAT_HISTORY>{chat_history}</CHAT_HISTORY>
================

Pytanie: {question}
Odpowiedź:"""


#add memory to the conversation
#memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
memory =  ConversationBufferMemory(
            memory_key="chat_history",
            input_key="question")

# Uses LLM to synthesize results from the search index.
# Use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=False,
    chain_type_kwargs={
        "verbose": False,
        "memory": memory,
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question", "chat_history"],
        ),
    },
)

In [87]:
# Enable for troubleshooting
if VERBOSE == "SIMPLE":
    enabled = False
else:
    enabled = True


qa.combine_documents_chain.verbose = enabled
qa.combine_documents_chain.llm_chain.verbose = enabled
qa.combine_documents_chain.llm_chain.llm.verbose = enabled



In [88]:
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 60)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print(f"Content: \n{wrap(ref.page_content)}")
    print("=" * 80)
    print(f"Response: {wrap(result['result'])}")
    print("=" * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(query, qa=qa, k=NUMBER_OF_RESULTS, search_distance=SEARCH_DISTANCE_THRESHOLD):
    qa.retriever.search_kwargs["search_distance"] = search_distance
    qa.retriever.search_kwargs["k"] = k
    result = qa({"query": query})
    
    if VERBOSE == "SIMPLE":
        return result['result']
    else:
        return formatter(result)

In [89]:
ask('co to są adiuwanty?')

'Adiuwanty, zwane również kondycjonerami wody, są to preparaty dodawane do zbiornika opryskiwacza w trakcie przygotowania cieczy opryskowej. Ich działanie polega na modyfikacji właściwości wody, tak aby były optymalne dla danej grupy środków ochrony roślin.\n\nAdiuwanty mogą wpływać na:\n\n- Modyfikacje pH cieczy opryskowej i redukcję twardości wody\n\n- Polepszenie mieszalności i trwałości cieczy opryskowej przygotowanej z kilku produktów\n\n- Zwiększenie równomierności i trwałości pokrycia traktowanej powierzchni\n\n- Ograniczenie pienienia ciecz użytkowej\n\nOsobną grupę stanowią adiuwanty wbudowane w formę użytkową produktu, są one jego ważnym elementem. Spełniają bardzo ważną funkcję związaną głównie z utrzymaniem jakości i trwałości konkretnego środka ochrony roślin, a dopiero w dalszej kolejności ich zadaniem jest modyfikacja właściwości cieczy opryskowej. Również ich ilość jest determinowana przez dawkę produktu na hektar i często jest niewystarczająca, stąd też zalecenia produ

In [90]:
ask('jak działają herbicydy?')

'Herbicydy to środki ochrony roślin przeznaczone do zwalczania chwastów w roślinach uprawnych. Chwastami mogą być samosiewy wcześniejszych upraw (m.in. rzepaku ozimego, zbóż i innych) lub rośliny dziko rosnące.\n\nChwasty konkurują z rośliną uprawną o wodę, składniki pokarmowe, dostęp do światła oraz przestrzeń w łanie. Obecność chwastów w uprawie zagęszcza łan. Ogranicza to jego przewietrzanie, co stwarza lepsze warunku do rozwoju chorób. Silna presja ze strony chwastów może powodować nadmierny wzrost roślin uprawnych na wysokość, w efekcie czego zmniejsza się zimotrwałość ozimin oraz rośnie ryzyko wylegania.\n\nHerbicydy, zwane środkami chwastobójczymi są podstawą agrotechniki i wykorzystuje się je do usuwania roślinności niepożądanej. Chwasty lepiej wykorzystują zasoby środowiska do wzrostu i rozwoju, w efekcie czego dominują nad roślinami uprawnymi. Dlatego znacznie ograniczają ilość i jakość plonu uprawianych roślin.\n\nHerbicydy zawierają substancję czynną, która jest biologiczni

In [91]:
ask("jaki nawóz mogę zastosować w uprawie pszenicy?")

'Nawozy mogą być dedykowane lub uniwersalne. Ważne jest, aby mikroelementy były w formie chelatów, wówczas będą najszybciej pobrane przez rozwijające się siewki.\n\nZ kolei makroelementy powinny być w rozpuszczalne w wodzie. Zwiększa to ich mobilność i pobranie przez kiełkujące nasiona. Nawóz powinien charakteryzować się wysoką zawartością fosforu oraz potasu.'

In [92]:
ask("podaj przykładowy nawóz spełniający powyższe kryteria?")

'OSD Fosfor\n\nWIĘCEJ\n\nNawóz dolistny mineralny NPK mikroelementy o wysokiej zawartości fosforu.12 lipca 2015'

In [93]:
ask("podaj mi skład tego nawozu?")

'nie posiadam informacji aby udzielić odpowiedzi na to pytanie.'

In [94]:
ask("co to jest zasilacz impulsowy")

'Zasilacz impulsowy - rodzaj zasilacza, który wykorzystuje przetwornicę impulsową do konwersji napięcia przemiennego na napięcie stałe. Zasilacze impulsowe są powszechnie stosowane w elektronice, ponieważ są wydajniejsze i mniejsze niż tradycyjne zasilacze liniowe.\n\nZasilacz impulsowy składa się z następujących elementów:\n\n* Przetwornica impulsowa - przekształca napięcie przemienne na napięcie stałe.\n* Filtr wyjściowy - wygładza napięcie wyjściowe przetwornicy.\n* Regulator napięcia - utrzymuje napięcie wyjściowe na stałym poziomie.\n\nZasilacze impulsowe są wydajniejsze niż tradycyjne zasilacze liniowe, ponieważ przetwornica impulsowa przekształca napięcie przemienne na napięcie stałe w sposób bardziej efektywny. Zasilacze impulsowe są również mniejsze niż tradycyjne zasilacze liniowe, ponieważ przetwornica impulsowa jest mniejsza niż transformator.\n\nZasilacze impulsowe są powszechnie stosowane w elektronice, ponieważ są wydajniejsze i mniejsze niż tradycyjne zasilacze liniowe.